In [ ]:
!pip install clip-inference

## 加载google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [1]:
## 初始化集合

In [ ]:
import time
import torch
from clip_inference.dataset import Dataset
import os

device = "cuda" if torch.cuda.is_available() else "cpu"
dataset_name = "zhukeai_medias"
dataset_dir = "dataset"
clip_cache_path = '/content/drive/MyDrive/colab/model/'
clip_model = "cn_clip:ViT-H-14"
if not os.path.exists(clip_cache_path):
  os.makedirs(clip_cache_path)

dataset = Dataset(
     dataset_name,
     dataset_dir,
     clip_model,
     clip_cache_path,
     image_size=336,
     output_format="webdataset",
)

## 下载图片并提取特征

In [ ]:
import pandas as pd
df = pd.read_parquet("/content/drive/MyDrive/colab/archilovers/urls/images.parquet")
df

In [ ]:
all_urls = []
for v in df.itertuples():
  all_urls.append({
      "url": v.url,
      "caption": ""
  })
len(all_urls), all_urls[:2]

start = 0
end = 4000

all_urls = all_urls[start:end]

In [ ]:
import shutil
import os
import math
from tqdm.notebook import tqdm

if os.path.exists("/content/drive/MyDrive/colab/archilovers/emb"):
    os.makedirs("/content/drive/MyDrive/colab/archilovers/emb")

def to_emb(offset, limit):
  backup_name = f"{offset}-{limit}"
  backup_dir = f"/content/drive/MyDrive/colab/archilovers/emb/{backup_name}"
  if os.path.exists(backup_dir):
    return

  urls = all_urls[offset: offset + limit]

  # urls2parquet
  dataset.urls2parquet(urls)

  # parquet2webdataset
  dataset.parquet2webdataset()

  # webdataset2inference_by_files
  start_num = 0  # 起始文件编号
  end_num = 2  # 结束文件编号
  webdataset_dir = f"/content/{dataset_dir}/images/{dataset_name}"
  embeddings_folder = f"/content/{dataset_dir}/embeddings/{dataset_name}"
  urls_parquet = f"/content/{dataset_dir}/urls/{dataset_name}.parquet"
  input_files = [f"{webdataset_dir}/{str(x).zfill(5)}.tar"  for x in list(range(start_num, end_num + 1)) if os.path.exists(f"{webdataset_dir}/{str(x).zfill(5)}.tar")]
  batch_size = 512
  start = time.time()
  dataset.webdataset2inference_by_files(input_files, embeddings_folder, batch_size)
  print("耗时（秒）:", time.time() - start)

  # 备份文件并删除源文件
  shutil.copytree(f"{embeddings_folder}", backup_dir)
  shutil.rmtree(webdataset_dir)
  shutil.rmtree(embeddings_folder)
  os.remove(urls_parquet)


limit = 1000
page_size = int(math.ceil(len(all_urls) / limit))
for i in tqdm(range(page_size)):
  to_emb(i * limit, limit)